In [10]:
import torch
from torch import nn 
from torch import optim
from river import compose, metrics, preprocessing, stream, anomaly, linear_model, datasets, compose
from IncrementalTorch.anomaly.anomaly import TorchAE, SklearnAnomalyDetector
from tqdm import tqdm
import river  
import torchvision
from pprint import pprint

from OnlineTorch.classifier import PyTorch2RiverClassifier
from torch import nn, optim

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDOneClassSVM

N_SAMPLES = 1_000
SEED = 42
track_name = "RBF"
LOSS = nn.BCELoss
OPTIMIZER = optim.AdamW
BATCH_SIZE=1
LEARNING_RATE=1e-3
METRIC = river.metrics.ROCAUC()


## Dataset

In [22]:
data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), 1000, seed=20)

# Autoencoder
## Undercomplete Autoencoder

In [39]:
def undercomplete_ae(n_features, latent_dim=1):
    net = nn.Sequential(
        nn.Dropout(),
        nn.Linear(n_features, 20), 
        nn.LeakyReLU(),
        nn.Linear(20, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, 20),
        nn.LeakyReLU(), 
        nn.Linear(20, n_features),
        nn.Linear(n_features,1),
        nn.Softmax()
    )
    return net

In [40]:
model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn = undercomplete_ae,
                loss_fn = LOSS,
                optimizer_fn = OPTIMIZER,
                batch_size=BATCH_SIZE,
                learning_rate=1e-3,
    )
)

In [41]:
data_stream = stream.shuffle(river.datasets.CreditCard().take(8000), n_samples, seed=42)
for x, y in data_stream:
    y_pred = model.predict_proba_one(x) #ruft learn_unsupervised auf, müssen wir learn_one dann überhaupt auch aufrufen?
    model.learn_one(x, y)
    METRIC.update(y, y_pred)
METRIC

ROCAUC: 0.877353

## Sparse Autoencoder

# Random

In [23]:
dataset = datasets.synth.RandomRBF(seed_model=7, seed_sample=seed,n_classes=10,n_features=200).take(n_samples)

def build_fn(n_features):
    net = nn.Sequential(
        nn.Linear(n_features, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 1),
        nn.Softmax()
    )
    return net

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn=build_fn,
                loss_fn=nn.BCELoss,
                optimizer_fn=optim.Adam,
                batch_size=1,
                learning_rate=1e-3,
    )
)

for x, y in data_stream:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    METRIC.update(y, y_pred)
METRIC

C:\Users\Manuel\Anaconda3\envs\rwch\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ROCAUC: 0.897878